## Loading and preprocessing data

## Plot Raw Samples

In [ ]:
# we will work with gaze data from the first recording
gaze = dataset.gaze[0]

# extract horizontal and vertical coordinates from the position column
df = gaze.samples

df = df.with_columns(
    [
        pl.col('position').list.get(0).alias('pos_x'),
        pl.col('position').list.get(1).alias('pos_y'),
    ]
)

# Assign back
gaze.samples = df

In [ ]:
pm.plotting.tsplot(
    gaze,
    channels=['pos_x', 'pos_y'],
    # Set separate y-axis for each channel.
    share_y=False,
    line_color='darkblue',
)

The {py:func}`~pymovements.plotting.traceplot` function visualizes the raw gaze samples as a continuous trajectory across the stimulus. In a traceplot, each gaze sample is connected in temporal order, showing how the point of regard moves over time.

Traceplots are useful for:
- Verifying that gaze data have been parsed and aligned correctly.
- Exploring viewing behavior across conditions or participants.
- Identifying artifacts or data quality issues.

A basic traceplot can be created with only a {py:class}`~pymovements.Gaze` object:

## Detecting and Visualizing Events

### Fixations

We can detect fixations by applying the I-VT or the I-DT method. 

The **I-VT (Velocity-Threshold Identification)** method distinguishes fixation and saccade points based on their point-to-point velocities. Each point is classified as a fixation if its velocity is below the specified threshold. Consecutive fixation points are then merged into a single fixation. A threshold of 20 degrees/second is commonly used as a default maximum value. Read more about the IVT method in the documentation: {py:func}`pymovements.events.detection.ivt`. 

The **I-DT (Dispersion-Threshold Identification)** method finds fixations by grouping consecutive points within a maximum separation (dispersion) threshold and a minimum duration threshold. The algorithm slides a moving window across the data: if the dispersion within the window is below the threshold, the window represents a fixation and is gradually expanded until the dispersion exceeds the threshold.
Read more about our implementation of the IDT method: {py:func}`pymovements.events.detection.idt`.

In [ ]:
# Detect fixations with a stricter threshold (1.0 degrees)
dataset.detect_events('idt', dispersion_threshold=1.0, name='fixation_1.0_idt')

# Detect fixations with a standard threshold (2.7 degrees)
dataset.detect_events('idt', dispersion_threshold=2.7, name='fixation_2.7_idt')

In [ ]:
# Compute fixation locations using pixel coordinates
dataset.compute_event_properties(('location', {'position_column': 'pixel'}))

The {py:func}`~pymovements.plotting.scanpathplot` function visualizes the sequence of fixations as circles placed at their spatial locations, with circle size indicating fixation duration. Each fixation has an arrow pointing to the next fixation in viewing order.

In [ ]:
pm.plotting.scanpathplot(gaze, event_name='fixation_1.0_idt')

By default, arrows are curved to prevent overlap with the stimulus (e.g., text). The curvature of the arrows is controlled by the `arrow_rad` parameter. Setting `arrow_rad` to zero disables the curvature and results in straight arrows, as shown below.

We can create an enhanced visualization by overlaying the scanpath plot with the traceplot. This shows both the fixations, their duration, and the raw gaze trajectory.

### Heatmap Plotting

In [ ]:
pm.plotting.heatmap(gaze)

In [ ]:
fig, ax = pm.plotting.heatmap(
    gaze=gaze,
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Interpolation On',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[10, 10],
)

In [ ]:
fig, ax = pm.plotting.heatmap(
    gaze,
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Interpolation Off',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[10, 10],
    interpolation='none',
)

In [ ]:
fig, ax = pm.plotting.heatmap(
    dataset.gaze[5],
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Higher Grid Size',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[25, 25],
)

In [ ]:
# detect saccades using the microsaccades algorithm
dataset.detect_events('microsaccades', minimum_duration=6, threshold_factor=6)

### Plotting the Saccadic Main Sequence

In [ ]:
# show the first three event dataframes.
# note that you can adjust the styling of the plot, e.g. setting a low
# alpha value allows you to change transparency to see overlapping data points
for event_df in dataset.events[:3]:
    pm.plotting.main_sequence_plot(
        event_df,
        event_name='saccade',
        fit=True,
        title='Main sequence plot',
        marker='x',
        marker_size=30,
        marker_color='green',
        marker_alpha=0.5,
    )